In [2]:
import torch
!pip install accelerate


In [3]:
with open("Data_BGE.txt", "r", encoding="utf-8") as f:
  text = f.read()

In [4]:
chars = sorted(list(set(text)))

In [5]:
print("".join(chars))


 !"'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzÆÉàâäæéêïôöüŒœάέήίαβγδεζηθικλμνξοπρςστυφχωόἀἄἆἐἑἰὀὖ᾽ῐῑ–—‘’“”﻿


In [6]:
stringtointeger = {}
integertostring = {}
for integer in range(len(chars)):
  integertostring[integer] = chars[integer]
  stringtointeger[chars[integer]] = integer

def encode(s):
    result = []
    for c in s:
        integer_value = stringtointeger[c]
        result.append(integer_value)
    return result

def decode(l_integers):
  result = []
  for num in l_integers:
    result.append(integertostring[num])
  return "".join(result)

print(encode("hello world"))
print(decode(encode("hello world")))

[60, 57, 64, 64, 67, 1, 75, 67, 70, 64, 56]
hello world


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:1000])

tensor([139,  39,  41,  28,  29,  24,  26,  28,   0,  42,  44,  39,  39,  38,
         42,  32,  37,  30,   1,  72,  60,  53,  72,   1,  43,  70,  73,  72,
         60,   1,  61,  71,   1,  53,   1,  75,  67,  65,  53,  66, 134,  75,
         60,  53,  72,   1,  72,  60,  57,  66,  23,   1,  32,  71,   1,  72,
         60,  57,  70,  57,   1,  66,  67,  72,   1,  59,  70,  67,  73,  66,
         56,   1,  58,  67,  70,   1,  71,  73,  71,  68,  57,  55,  72,  61,
         66,  59,   1,  72,  60,  53,  72,   1,  53,  64,  64,   1,  68,  60,
         61,  64,  67,  71,  67,  68,  60,  57,  70,  71,   7,   1,  61,  66,
          1,  71,  67,   1,  58,  53,  70,   1,  53,  71,   1,  72,  60,  57,
         77,   1,  60,  53,  74,  57,   1,  54,  57,  57,  66,   1,  56,  67,
         59,  65,  53,  72,  61,  71,  72,  71,   7,   1,  60,  53,  74,  57,
          1,  58,  53,  61,  64,  57,  56,   1,  72,  67,   1,  73,  66,  56,
         57,  70,  71,  72,  53,  66,  56,   1,  75,  67,  65,  

In [8]:
n = int(0.9*len(data)) #Use 90 percent for training and the 10 percent data for validation
training_data = data[:n]
validation_data = data[n:]

In [9]:
context_size = 8
print(training_data[:context_size+1])

tensor([139,  39,  41,  28,  29,  24,  26,  28,   0])


In [10]:
input_data = training_data[:context_size]
expected_output = training_data[1:context_size+1]
for i in range(context_size):
  context = input_data[:i+1]
  target = expected_output[i]
  print(f"input {context} ouput {target}")

input tensor([139]) ouput 39
input tensor([139,  39]) ouput 41
input tensor([139,  39,  41]) ouput 28
input tensor([139,  39,  41,  28]) ouput 29
input tensor([139,  39,  41,  28,  29]) ouput 24
input tensor([139,  39,  41,  28,  29,  24]) ouput 26
input tensor([139,  39,  41,  28,  29,  24,  26]) ouput 28
input tensor([139,  39,  41,  28,  29,  24,  26,  28]) ouput 0


In [11]:
batch_size = 6
context_size = 8
def get_batch(data_type):
    data = training_data if data_type == "training_data" else validation_data
    index_for_batches = torch.randint(len(data)-context_size, (batch_size,))
    print(index_for_batches)
    x = torch.stack([data[index:index+context_size] for index in index_for_batches])
    y = torch.stack([data[index+1:index+context_size+1] for index in index_for_batches])
    return x, y


In [12]:
input, output = get_batch("traning_data")

tensor([ 57979, 140986,  79898,  48462,   6370,  49490])


In [13]:
print("input", input.shape, "output", output.shape)

input torch.Size([6, 8]) output torch.Size([6, 8])


In [14]:
for row in range(batch_size): #to acesss the each tensor at each row
    for col in range(context_size): #to access each tokenize value at each column
        input_to_model_context =  input[row, :col+1] #each row upto the col+1 as col+1 isnt included
        print(output[row,col])
        expect_y = output[row,col] # specfic word not a list
        print("input_to_model_context", input_to_model_context, "expected_output", expect_y)
    

tensor(66)
input_to_model_context tensor([53]) expected_output tensor(66)
tensor(56)
input_to_model_context tensor([53, 66]) expected_output tensor(56)
tensor(1)
input_to_model_context tensor([53, 66, 56]) expected_output tensor(1)
tensor(67)
input_to_model_context tensor([53, 66, 56,  1]) expected_output tensor(67)
tensor(58)
input_to_model_context tensor([53, 66, 56,  1, 67]) expected_output tensor(58)
tensor(1)
input_to_model_context tensor([53, 66, 56,  1, 67, 58]) expected_output tensor(1)
tensor(37)
input_to_model_context tensor([53, 66, 56,  1, 67, 58,  1]) expected_output tensor(37)
tensor(53)
input_to_model_context tensor([53, 66, 56,  1, 67, 58,  1, 37]) expected_output tensor(53)
tensor(72)
input_to_model_context tensor([55]) expected_output tensor(72)
tensor(53)
input_to_model_context tensor([55, 72]) expected_output tensor(53)
tensor(72)
input_to_model_context tensor([55, 72, 53]) expected_output tensor(72)
tensor(67)
input_to_model_context tensor([55, 72, 53, 72]) expecte

In [16]:
Batch, Token, Channel = 4,8,2
x = torch.randn(Batch, Token, Channel)
print(x.shape)

torch.Size([4, 8, 2])


In [ ]:
xbagofwords = torch.zeros((Batch, Token, Channel)) #here we want to have some form of communication between the token.in this case we would take average all the number before it"""

for b in range(Batch):
    for t in range(Token):
        xprev = x[b, :t+1]   #in the b row upto t, including t
        xbagofwords[b,t] = torch.mean(xprev,0) #here at the position t in some random batch b we have average all the previous numbers. Like attention.
print(x[0])

if you have a tensor x with the shape (1, 3, 2), the data might look something like this:

x = torch.tensor([[[1, 2], [3, 4], [5, 6]]])

You have 1 batch (Batch = 1).
Each batch contains 3 tokens (Token = 3).
Each token is represented by a 2-dimensional vector (or 2 features per token, Channel = 2).
In this tensor:

The first dimension (Batch) has only one element.
The second dimension (Token) represents the three tokens.
The third dimension (Channel) represents the two features (or channels) for each token.
When you refer to [1, 3, 5] and [2, 4, 6], you're essentially talking about the two features (channels) across the three tokens. Here's how they are distributed:

The first feature (channel) across the three tokens is [1, 3, 5].
The second feature (channel) across the three tokens is [2, 4, 6].

For each token t, you're selecting all tokens up to and including the current one (x[b, :t+1]), and then computing the mean along the first dimension (Token).
This mean is calculated separately for each feature (channel). So, for the first token, the mean would just be the features of the first token itself. For the second token, it would be the mean of the features of the first and second tokens, and so on.
So, in the context of calculating the mean:

When t = 0, xprev is [[1, 2]], and the mean is [1, 2] (since there's only one token to consider).
When t = 1, xprev is [[1, 2], [3, 4]], and the mean for each feature would be:
First feature: mean of [1, 3] -> 2
Second feature: mean of [2, 4] -> 3
The result for t = 1 would be [2, 3], and so on for larger t.

In [33]:
print(xbagofwords[0])

tensor([[-1.3333,  0.8926],
        [-1.0479,  0.3544],
        [-0.9779, -0.2347],
        [-0.9516, -0.1231],
        [-0.4728, -0.0203],
        [-0.2053,  0.0659],
        [-0.2575,  0.1247],
        [-0.0109,  0.1433]])


The method of allowing each token to access information from others can be extremely inefficient, particularly when considering the O(n^2) complexity involved in the multiplication process. Let's explore how we can optimize this using matrix multiplication.

In [36]:
simple_matrix = torch.ones(3,3)
matrix_2 = torch.randint(0,10,(3,2)).float() 
c = simple_matrix @ matrix_2
print("matrix_2",matrix_2, "simple_matrix", simple_matrix, "c", c)

matrix_2 tensor([[1., 0.],
        [5., 5.],
        [7., 2.]]) simple_matrix tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]) c tensor([[13.,  7.],
        [13.,  7.],
        [13.,  7.]])


now what if we define simple_matrix as identity matrix as

In [37]:
simple_matrix = torch.tril(torch.ones(3,3))

In [38]:
print(simple_matrix)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])


In [40]:
simple_matrix = torch.tril(torch.ones(3,3))
matrix_2 = torch.randint(0,10,(3,2)).float() 
c = simple_matrix @ matrix_2
print(c)

tensor([[ 4.,  7.],
        [12., 11.],
        [13., 16.]])


This form of skew identity matrix will help us in accumulating sum. but we need the mean not the averge not the sum. Lets see how we can do that.

In [47]:
simple_matrix = torch.tril(torch.ones(3, 3))
simple_matrix = simple_matrix / torch.sum(simple_matrix, 1, keepdim=True)
matrix_2 = torch.randint(0,10,(3,2)).float() 
c = simple_matrix @ matrix_2
print(matrix_2, c)

tensor([[4., 9.],
        [3., 2.],
        [1., 8.]]) tensor([[4.0000, 9.0000],
        [3.5000, 5.5000],
        [2.6667, 6.3333]])


let's get back to our orginal example now
xbagofwords = torch.zeros((Batch, Token, Channel)) #here we want to have some form of communication between the token.in this case we would take average all the number before it"""

for b in range(Batch):
    for t in range(Token):
        xprev = x[b, :t+1]   #in the b row upto t, including t
        xbagofwords[b,t] = torch.mean(xprev,0) #here at the position t in some random batch b we have average all the previous numbers. Like attention.
print(x[0])

In [51]:
identity_matrix = torch.tril(torch.ones(Token,Token))
identity_matrix = identity_matrix / torch.sum(identity_matrix, 1, keepdim=True)
xbagofwords_e = identity_matrix @ x # (B, T, T) @ (B, T, C)
torch.allclose(xbagofwords, xbagofwords_e)

True

In [71]:
tril = torch.tril(torch.ones(Token,Token))
print(tril)
identity_matrix = torch.zeros((Token,Token))
print(identity_matrix)

identity_matrix = identity_matrix.masked_fill(tril == 0, float('-inf'))
#this make sure that no forward communication happens between the tokens
print(identity_matrix)
identity_matrix = F.softmax(identity_matrix, dim=-1) # exponentiate and divide by the sum of exponent will give us probablility
print(identity_matrix)
xbagofwords_3 = identity_matrix @ x
torch.allclose(xbagofwords, xbagofwords_3)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0.,

True

Let's move on to the crux of "Attention is all you need" paper. i.e self attention

In [75]:
Batch, Tokens, Channels = 4, 8, 32
x = torch.randn(Batch,Token,Channels)
tril = torch.tril(torch.ones(Tokens, Tokens)) 
identity_matrix = torch.zeros((Token,Token))


identity_matrix = identity_matrix.masked_fill(tril == 0, float('-inf'))

identity_matrix = F.softmax(identity_matrix, dim=-1) # exponentiate and divide by the sum of exponent will give us probablility

output = identity_matrix @ x
output.shape

torch.Size([4, 8, 32])

Right now, we're essentially averaging all the last tokens in each batch at every point. But remember, there's no forward communication. This is thanks to how we've set up the '-inf' in the identity_matrix. However, based on the context (the data), this averaging shouldn't just be uniform. Instead, it should be weighted according to the data.

Let's dive deeper into how the key, query, and value really work at the core. Picture this: every token in each batch emits two things – a query and a key. Think of a query as asking, 'What am I looking for?' and a key as saying, 'Here’s what I represent.' So, each token broadcasts its query to all preceding tokens (not the ones ahead, remember the infinity setup) in the same batch, essentially saying, 'I'm searching for this info.' This call is answered by the last tokens through their keys. We use the dot product as a measure of how similar the query and the key are. A larger dot product value indicates greater similarity. This similarity score gets stored in the identity matrix.

In [78]:
Batch, Tokens, Channels = 4, 8, 32

x = torch.randn(Batch,Token,Channels)

#single head self-attention
head_size = 16
key = nn.Linear(Channels, head_size, bias=False)
query = nn.Linear(Channels, head_size, bias=False)
value = nn.Linear(Channels, head_size, bias=False)
key = key(x)
query = query(x)
identity_matrix = query @ key.transpose(-2, -1) #(B,T,16) @ (B,16,T) ----> (B, T, T) for each Batch we have T, T matrix


tril = torch.tril(torch.ones(Tokens, Tokens)) 
identity_matrix = torch.zeros((Token,Token))


identity_matrix = identity_matrix.masked_fill(tril == 0, float('-inf'))

identity_matrix = F.softmax(identity_matrix, dim=-1) # exponentiate and divide by the sum of exponent will give us probablility
v = value(x)
output = identity_matrix @ v
output.shape

torch.Size([4, 8, 16])

In the case of cross-attention, as utilized in the encoder-decoder architecture of Transformer models, it is necessary to omit the
use of float('-inf'). This adjustment is important because, in cross-attention, tokens are allowed to communicate with each other, 
which is a key aspect of how the encoder and decoder interact. This approach differs from the self-attention mechanism in the encoder, 
where the use of float('-inf') is applied to prevent forward-looking information flow. The original Transformer paper, which introduced 
the encoder-decoder architecture, implements this strategy to facilitate effective communication between tokens across the encoder and 
decoder.